In [ ]:
import os
from typing import Dict, Any, List
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_groq import ChatGroq
from langchain.tools import tool
from langgraph.graph import StateGraph, END, MessagesState
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun
import operator
from google.colab import userdata
import operator

!pip install ddgs

# Get API key from environment variable
from google.colab import userdata
import os

try:
    groq_key = userdata.get('GROQ_API_KEY')
    os.environ['GROQ_API_KEY'] = groq_key
except Exception as e:
    raise ValueError(f"Failed to load GROQ_API_KEY secret: {e}")
# set up llm with groq
llm = ChatGroq(model="llama-3.3-70b-versatile",
                 temperature=0.7,
                 max_tokens=1024)

# define tools
search_tool = DuckDuckGoSearchRun()
# to search web and return results

repl = PythonREPL()

# python repl for code execution and save results and errors
@tool
def python_repl_tool(code: str) -> str:
    """Execute Python code and return the output."""
    try:
        result = repl.run(code)
        return str(result)
    except Exception as e:
        return f"Error executing code: {str(e)}"


# tool 3: evaluate mathematical expresssionsusing repl
@tool #makes these functions callable by llm as tools
def calculator(expression : str) -> str:
    """Evaluate a mathematical expression and return the result."""
    try:
        # Safely evaluate the expression using eval with restricted globals
        allowed_ops = {
            "add" : operator.add,
            "sub" : operator.sub,
            "mul" : operator.mul,
            "div" : operator.truediv,
            "pow" : operator.pow
        }
        result = repl.run(f"print({expression})")
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

tools = [search_tool, python_repl_tool, calculator]

# bind above 3 tools to llm using bind_tools method
llm_with_tools = llm.bind_tools(tools)

# react prompt template
react_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are an intelligent assistant that can use tools to answer questions step by step."),
        MessagesPlaceholder(variable_name="history"), # to hold messsages history
        HumanMessage(content=(
            "use this format:\n"
            "Thought: <your thought process>\n"
            "Action: <the tool you want to use>\n"
            "Observation: <the result from the tool>\n"
            "...(repeat Thought/Action/Observation as needed)\n"
            "Final Answer: [answer]"
        )),
    ]
)


"""
system prompt sets agents role
messages placeholder is where the history of interactions will be stored
human message provides instructions to the agent on how to use tools and format its response
"""


# create agent with langraph ( for memory and looping )
class AgentState(MessagesState):
    """state for the agent to hold messages history and tool outputs"""
    pass

def agent_node(state: AgentState) -> Dict[str, List[Any]]:
    """node that calls llm to decide next action"""
    result = llm_with_tools.invoke(state["messages"])
    if result.tool_calls: # if there are tool calls in results as element
        return {"messages" : [AIMessage(content=result.content, tool_calls = result.tool_calls)]}
    else:
        return {"messages" : [AIMessage(content=result.content)]}

# Tool execution node - manually execute tools
def tool_node(state: AgentState) -> Dict[str, List[Any]]:
    """node that executes tools and returns observations"""
    last_message = state["messages"][-1]

    tool_outputs = []

    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            tool_name = tool_call["type"] if "type" in tool_call else tool_call.get("tool_name", tool_call.get("name", ""))

            # Handle different tool call formats
            if isinstance(tool_call, dict):
                tool_name = tool_call.get("name") or tool_call.get("tool_name") or tool_call.get("type")
                tool_input = tool_call.get("args") or tool_call.get("input", {})
            else:
                tool_name = getattr(tool_call, "name", None)
                tool_input = getattr(tool_call, "args", {})

            # Execute the tool
            tool_result = None
            for available_tool in tools:
                if available_tool.name == tool_name:
                    try:
                        if isinstance(tool_input, dict):
                            tool_result = available_tool.invoke(tool_input)
                        else:
                            tool_result = available_tool.invoke({"input": tool_input})
                    except Exception as e:
                        tool_result = f"Error executing tool: {str(e)}"
                    break

            # Handle web search tool (returns string directly)
            if tool_name == "duckduckgo_search" or tool_name == "DuckDuckGoSearchRun":
                try:
                    # Modified to correctly extract 'query' if present, otherwise fall back to 'input'
                    search_query = tool_input.get("query") if isinstance(tool_input, dict) and "query" in tool_input else \
                                   (tool_input if isinstance(tool_input, str) else tool_input.get("input", ""))
                    tool_result = search_tool.run(search_query)
                except Exception as e:
                    tool_result = f"Error: {str(e)}"

            if tool_result is None:
                tool_result = "Tool not found"

            tool_outputs.append(ToolMessage(content=str(tool_result), tool_call_id=tool_call.get("id", tool_name)))

    return {"messages": tool_outputs}

# graph to connect agent and tool nodes
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tool", tool_node)
workflow.set_entry_point("agent")

"""
starts at agent node
loops to tools if needed
then back to agent
ends when no tools are called

it allows cyclic workflow
"""

# process if tool calls
def should_continue(state: AgentState) -> str:
    last_message = state["messages"][-1] # get last message in history
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:# if last message has tool_calls then continue
        return "tool"
    else:
        return "end"

workflow.add_conditional_edges("agent", should_continue, {"tool": "tool", "end": END}) # if agent_node has tools then continue else end
workflow.add_edge("tool", "agent") # after tool execution go back to agent node

# add memory
# memory = MemorySaver() # to save state of graph after each step - not available in current langgraph version
graph = workflow.compile()

# agent executor function to run the graph
def run_agent(query: str, thread_id : str = "default") -> str:
    """ run the agent with memory for given thread id"""
    config = {"configurable" : {"thread_id" : thread_id}} # to identify different conversations
    inputs = {"messages" : [HumanMessage(content=query)]}

    # stream the responses to show step by step
    reasoning_trace = []
    try:
        for event in graph.stream(inputs, config=config, stream_mode="values"):
            if "messages" in event:
                last_message = event["messages"][-1]
                # if last_message is aimessage
                if isinstance(last_message, AIMessage):
                    if last_message.content:
                        reasoning_trace.append(last_message.content)
                        print("Agent's response: ", last_message.content)
                    # if tool calls are present
                    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
                        for tool_call in last_message.tool_calls:
                            tool_name = tool_call.get("name") or tool_call.get("type")
                            tool_args = tool_call.get("args") or tool_call.get("input")
                            reasoning_trace.append(f"Tool called: {tool_name} with input {tool_args}")
                            print(f"Tool called: {tool_name} with input {tool_args}")
                elif isinstance(last_message, ToolMessage):
                    reasoning_trace.append(f"Tool result: {last_message.content}")
                    print(f"Tool result: {last_message.content}")
                elif isinstance(last_message, (tuple, list)) and len(last_message) and hasattr(last_message[0], "content"): # if last message is a list of messages
                    reasoning_trace.append(f"observation: {last_message[0].content}")

        # final answer is last AImessage content
        final_answer = "No answer generated"
        for msg in reversed(event["messages"]):
            if isinstance(msg, AIMessage) and msg.content:
                final_answer = msg.content
                break

        print("\nReasoning trace:")
        for step in reasoning_trace:
            print(step)

        return final_answer
    except Exception as e:
        print(f"Error running agent: {str(e)}")
        return f"Error: {str(e)}"

if __name__ == "__main__":
    # test the agent
    query = "What is 2 + 2?"
    print("Query: ", query)
    answer = run_agent(query, thread_id="test1")
    print("Final Answer: ", answer)


    # follow up with memory
    follow_up_query = "What about 3 + 5?"
    print("\nFollow up Query: ", follow_up_query)
    follow_up_answer = run_agent(follow_up_query, thread_id="test1") # same thread id to access memory of previous conversation
    print("Final Answer: ", follow_up_answer)

    # web serach example
    web_query = "What is the capital of France?"
    print("\nWeb Query: ", web_query)
    web_answer = run_agent(web_query, thread_id="test2") # different thread id which means different conversation without memory of previous one
    print("Final Answer: ", web_answer)

    # code execution example
    code_query = "Write a python function to calculate factorial of a number and calculate factorial of 5."
    print("\nCode Query: ", code_query)
    code_answer = run_agent(code_query, thread_id="test3") # different thread id which means different conversation without memory of previous one
    print("Final Answer: ", code_answer)




Query:  What is 2 + 2?
Tool called: calculator with input {'expression': '2 + 2'}
Tool result: 4

Agent's response:  The answer to 2 + 2 is 4.

Reasoning trace:
Tool called: calculator with input {'expression': '2 + 2'}
Tool result: 4

The answer to 2 + 2 is 4.
Final Answer:  The answer to 2 + 2 is 4.

Follow up Query:  What about 3 + 5?
Tool called: calculator with input {'expression': '3 + 5'}
Tool result: 8

Agent's response:  The answer to 3 + 5 is 8.

Reasoning trace:
Tool called: calculator with input {'expression': '3 + 5'}
Tool result: 8

The answer to 3 + 5 is 8.
Final Answer:  The answer to 3 + 5 is 8.

Web Query:  What is the capital of France?
Tool called: duckduckgo_search with input {'query': 'capital of France'}


Tool result: As the capital of France , Paris is the seat of France's national government. For the executive, the two chief officers each have their own official residences, which also serve as their offices. In France, the taxation of capital gains (plus-value) from the sale of works of art such as paintings by deceased artists (typically handled by heirs) in 2025-2026 follows the standard regime for movable property with no specific exemptions or major changes introduced for these years (proposals to integrate artworks into wealth tax were rejected). The two main options are: the forfaitaire regime (most common), which applies a flat tax of 6.5% (6% tax + 0.5% CRDS) on the full sale price if it exceeds €5,000, regardless of any actual gain or loss; or the irrevocable real regime (elected via form no. 2092 within one month of the sale), where the capital gain (sale price minus acquisition price or market value declared at succession) is taxed at 19% income tax plus 17.2% social levies

In [ ]:
!pip install langchain-experimental

In [ ]:
!pip install langchain-groq